# DTLZ 2:
Here we show that RVEA works better if we have limited function evaluations (5000 in this case) for dtlz .

# DTLZ 7
Here we show that even tho NSGAIII finds more solutions the few solutions that RVEA finds are better (hopefully have not checked numbers yet)

## Optimization process


In [16]:
import numpy as np
import pandas as pd
import math


import random
import scipy

from desdeo_tools.utilities.pmod import get_pmod
from desdeo_tools.utilities.pmda import get_pmda
import ehmetric as eh
from desdeo_tools.utilities.rmetric import RMetric



from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from desdeo_problem import variable_builder, MOProblem, VectorObjective
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from sklearn.preprocessing import Normalizer
from desdeo_emo.EAs import NSGAIII, RVEA, IOPIS_RVEA, IOPIS_NSGAIII, IBEA

from desdeo_emo.utilities import animate_init_, animate_next_

refpoints_dtlz2_l = [[0.10, 0.10, 0.94],[0.10, 0.94, 0.10],  [0.94, 0.10, 0.10], [0.7,0.6,0.55] ]
refpoints_dtlz2_d = [[0.20, 0.20, 0.85],[0.15, 0.25, 0.83],  [0.18, 0.22, 0.80], [0.23,0.25,0.75] ]
refpoints_dtlz3_l = [[21.2, 0.0, 52.9], [4e-32, 3.77, 9.43], [1.26e-32, 0.386, 2.32], [0.172, 0.0, 1.03]]
refpoints_dtlz7 = [[0.11, 0.10, 5.4 ],[0.70, 0.14, 4.50],  [0.76, 0.76, 3.5], [0.14, 0.70, 4.5] ]

refpoints = refpoints_dtlz7
minimization = [True]*3 # for eh-metric
rvea_solutions = []
nsga_solutions = []
RP = []
n_interactions = 4
n_obj = 3
n_var = 8
name = "DTLZ7"
problem = test_problem_builder(
    name=name, n_of_objectives=n_obj, n_of_variables=n_var
    )
evolver_rvea = RVEA(problem, n_gen_per_iter=1000, n_iterations=5, interact=True, total_function_evaluations= 50000000 )#total_function_evaluations= 9000000
evolver_nsga = NSGAIII(problem, n_gen_per_iter=1000, n_iterations=5, interact=True, total_function_evaluations= 5000000) #total_function_evaluations= 600000
#evolver_nsga.translation_param = 0.3

#evolver_nsga = IOPIS_RVEA(problem, n_gen_per_iter=500, n_iterations=1)
problem.ideal = np.asarray([0] * n_obj)
problem.ideal_fitness = problem.ideal
problem.nadir = abs(np.random.normal(size=n_obj, scale=0.15)) + 1

non_dominated = evolver_rvea.population.non_dominated_fitness()

for i in range(n_interactions):
    pref_rvea, plot_rvea = evolver_rvea.requests()
    pref_rvea[2].response = pd.DataFrame(
    [refpoints[i]], columns=pref_rvea[2].content["dimensions_data"].columns
    )
    pref_nsga, plot_nsga = evolver_nsga.requests()
    pref_nsga[2].response = pd.DataFrame(
    [refpoints[i]], columns=pref_nsga[2].content["dimensions_data"].columns
    )
    print(f"Running iteration {evolver_rvea._iteration_counter+1}")
    evolver_rvea.iterate(pref_rvea[2])
    evolver_nsga.iterate(pref_nsga[2]) #

    non_dominated_rvea = evolver_rvea.population.non_dominated_fitness()
    rvea_results = evolver_rvea.population.objectives[non_dominated_rvea]
    rvea_solutions.append(rvea_results)

    non_dominated_nsga = evolver_nsga.population.non_dominated_fitness()
    nsga_results = evolver_nsga.population.objectives[non_dominated_nsga]
    nsga_solutions.append(nsga_results)
    
    


Running iteration 1
Running iteration 2
Running iteration 3
Running iteration 4


# Visualization

In [17]:
figure_rvea = animate_init_(np.vstack((rvea_solutions[0],[1,1,1],[0,0,0])), filename="RVEAw"+name+".html")
figure_nsga = animate_init_(np.vstack((nsga_solutions[0],[1,1,1],[0,0,0])), filename="NSGA"+name+".html")


for i in range(n_interactions-1):
    figure_rvea = animate_next_(
            np.vstack((rvea_solutions[i+1], [1,1,1],[0,0,0])),
            figure_rvea,
            filename="RVEAw"+name+".html",
            generation=i+1,
        )
    figure_nsga = animate_next_(
            np.vstack((nsga_solutions[i+1], [1,1,1],[0,0,0])),
            figure_nsga,
            filename="NSGA"+name+".html",
            generation=i+1,
        )

Plot saved as:  RVEAwDTLZ7.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.
Plot saved as:  NSGADTLZ7.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.


# Calculating some indicators


In [18]:
for i in range(n_interactions):
    

    
    

    #R-metric EH-metric
    #normalization
    _max = np.max(np.vstack((rvea_solutions[i],nsga_solutions[i], refpoints[i])) , axis= 0)
    _min = np.min(np.vstack((rvea_solutions[i],nsga_solutions[i], refpoints[i])) , axis= 0)
    norm_rvea = (rvea_solutions[i] - _min)/ (_max - _min)
    norm_nsga = (nsga_solutions[i] - _min)/ (_max - _min)
    norm_RP = (refpoints[i] - _min)/ (_max - _min)

    areas, steps = eh.RunEHMetric([pd.DataFrame(norm_rvea), pd.DataFrame(norm_nsga)], norm_RP, minimization)
    
    rmetric_rvea = RMetric(problem, norm_RP, delta =0.2).calc(F = norm_rvea, others = norm_nsga)
    rmetric_nsga = RMetric(problem, norm_RP, delta = 0.2).calc(F = norm_nsga, others = norm_rvea)
    print("this is [RVEA, NSGA] R-metric:", [rmetric_rvea[1], rmetric_nsga[1]])
    print("this is [RVEA, NSGA] EH-metric:", areas)
    print("this is [RVEA, NSGA] pmod:", [get_pmod(refpoints[i],rvea_solutions[i],0.1,2), get_pmod(refpoints[i],nsga_solutions[i],0.1,2)])
    print("this is [RVEA, NSGA] pmda:", [get_pmda(refpoints[i],rvea_solutions[i],3,0.5,0.01), get_pmda(refpoints[i],nsga_solutions[i],3,0.5,0.01)])


this is [RVEA, NSGA] R-metric: [5.927247177416777, 6.66867390126505]
this is [RVEA, NSGA] EH-metric: [0.2760380836655655, 0.5611596623735252]
this is [RVEA, NSGA] pmod: [6.543197494663459, 7.8877931943452815]
this is [RVEA, NSGA] pmda: [4.632570595538658, 5.050253760447804]
this is [RVEA, NSGA] R-metric: [5.859375365912599, 6.40113308327709]
this is [RVEA, NSGA] EH-metric: [0.5687590881758553, 0.41939656715108586]
this is [RVEA, NSGA] pmod: [5.713521760525143, 6.62605315744862]
this is [RVEA, NSGA] pmda: [4.247907181694133, 5.091985915159812]
this is [RVEA, NSGA] R-metric: [7.30195257200826, 6.488536533865337]
this is [RVEA, NSGA] EH-metric: [0.6877447430243327, 0.12130005438302977]
this is [RVEA, NSGA] pmod: [5.4181341683346, 5.833147930535961]
this is [RVEA, NSGA] pmda: [3.226788332530852, 4.940112136197966]
this is [RVEA, NSGA] R-metric: [5.9089147552675385, 2.930107185783777]
this is [RVEA, NSGA] EH-metric: [0.4820169139927103, 0.19631767158801477]
this is [RVEA, NSGA] pmod: [5.690

In [14]:
np.savetxt("NS_dtlz7_3_in1.csv",nsga_solutions[0], delimiter=",")
np.savetxt("NS_dtlz7_3_in2.csv",nsga_solutions[1], delimiter=",")
np.savetxt("NS_dtlz7_3_in3.csv",nsga_solutions[0], delimiter=",")
np.savetxt("NS_dtlz7_3_in4.csv",nsga_solutions[1], delimiter=",")

## PMOD

## PMDA

ModuleNotFoundError: No module named 'bokeh'